# Generating music using LSTM Recurent Neural Networks (RNN)

The goal of this notebook will be to explore one method of generating music scores (in midi or another format) based on a corpus which can then be played by a midi player with a synthesizer.

inspiration for this project:
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5


### 1) Collecting MIDI files

In this project, we will be using not mp3 or wav files but MIDI files which take up less space and have lower dimensionality and are easier to work with.

For our project, we need to collect a large amount of MIDI files for a single instrument type in a single style (preferrably) which we will preprocess then use to feed into the LSTM in order to train it.

There are a large amount of databases where MIDI files are available:
- https://freemidi.org/genre-blues
- http://www.midiworld.com/search/1/?q=blues


### A) Web scrapping

We will be using python for web scrapping in order to retrieve all of those files as easily as possible (https://towardsdatascience.com/how-to-web-scrape-with-python-in-4-minutes-bc49186a8460)

### better tutorial for later, https://www.import.io/post/machine-learning-dataset-musical-training/
For now, lets just focus on a single artist (BB King) from the freemidi.org website

first, lets make sure that the correct packages are installed for the project to run properly

In [0]:
!pip install music21
!pip install glob
!pip install pickle

!pip install keras

  Could not find a version that satisfies the requirement glob (from versions: )
No matching distribution found for glob
  Could not find a version that satisfies the requirement pickle (from versions: )
No matching distribution found for pickle


### B) Loading MIDI files and pre-processing them a little.

We now have a handfull of midi files. We can load them all at once or by batches depending on memory constraints that we may have. However, since we have only a few files we will not worry about this yet.

These files usually contain the tracks of multiple instruments that are being played at the same time. For the scope of our project, we only want to keep the guitar/piano part of these tracks.

The library we will be using is called Music21 to both read and process the files.

https://github.com/PacktPublishing/Python-Deep-Learning-Projects/tree/master/Chapter06

In [0]:
from music21 import converter,instrument, note , chord
import glob
import pickle

In [0]:
instruments = [instrument.ElectricGuitar(),instrument.Piano(),instrument.Guitar()]


for instru in instruments:
    print(type(instru))

<class 'music21.instrument.ElectricGuitar'>
<class 'music21.instrument.Piano'>
<class 'music21.instrument.Guitar'>


In [0]:
def parse_notes(notes_to_parse,notes):
    """parses notes in the track sent to the function, it will add notes and vectorize the chords
        we also add rests if they are present in the part
    """
    
    for element in notes_to_parse:
        
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
        elif isinstance(element, note.Rest):
            notes.append("rest")

    return notes

for loading the midi files into the notebook without any problems:

https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92


In [0]:
# donner acces à drive au dossier ou se trouve les fichiers midi

import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

os.chdir("/content/drive/My Drive/Création Numérique/S2/code")
!ls

Mounted at /content/drive
 data			       notes	        trained_models2
 generated_music	       old_data         trained_models_piano
'LSTM music generator.ipynb'   test.ipynb
'MIDI scrapping V2.ipynb'      trained_models


In [0]:
# array that will store all of the notes in our songs
notes = []

#instrument classes that we want our loop to store in the notes array
instruments = [instrument.ElectricGuitar()]


# iterates over all midi files in the data directory
for file in glob.glob("data/*.mid"):

    notes_to_parse = []
    try:
        # parse midi file
        midi = converter.parse(file)
        tracks = instrument.partitionByInstrument(midi)
        print("parsing file:",file)
        
        if len(tracks.parts) > 1:
            
            for i, part in enumerate(tracks.parts):
                print(part[0])
                for instru in instruments:
                    if isinstance(part[0], type(instru)):
                        # we want to keep that part for our training data
                        print(part[0], "=", instru)
                        try:
                            notes = parse_notes(part.recurse(),notes)
                        except Exception as e:
                            print("error parsing file")
                            print(e)
        else:
            # only one track, keep this one
            print("single track", tracks.parts[0])
            notes = parse_notes(tracks.parts.recurse(),notes)

        # if the file cannot be opened properly
    except Exception as e:
        print("unable to open file:", file)
        print(e)
         


if len(notes)>= 1:
    
    print("recovered",len(notes),"different tracks")
    with open('data/notes/note_tab', 'wb') as filepath:
            pickle.dump(notes, filepath)
else:
    print("no notes recovered")  

unable to open file: data/BillieMyers.mid
badly formated midi bytes, got: b''
parsing file: data/CallItStormyMonday.mid
Flute
Electric Guitar
Electric Guitar = Electric Guitar
Trumpet
Tenor Saxophone
Saxophone
Acoustic Bass
Violin
parsing file: data/DanceWithMeHenry.mid
Trombone
Tenor Saxophone
Piano
Electric Guitar
Electric Guitar = Electric Guitar
StringInstrument
Fretless Bass
Acoustic Bass
parsing file: data/BlueSky.mid
Fretless Bass
Piccolo

Electric Guitar
Electric Guitar = Electric Guitar
Piano
parsing file: data/DontAnswerTheDoor.mid
Electric Guitar
Electric Guitar = Electric Guitar
Fretless Bass

Flute
parsing file: data/EverythingIDoIsWrong.mid

Harmonica
parsing file: data/DarlingYouKnowILoveYou.mid
Electric Guitar
Electric Guitar = Electric Guitar

parsing file: data/DearDiary.mid

Acoustic Bass
Piccolo
Ocarina
Guitar
Piano
Horn
parsing file: data/Dreams.mid

Saxophone
Electric Guitar
Electric Guitar = Electric Guitar
Piano
parsing file: data/Emilys.mid

StringInstrument
pa

FileNotFoundError: ignored

### one hot encoding data

The data is in a better and easier to understand format now but it is still not enough. Indeed, we need to feed a vector as an input to the network. We cannot just input the data as is into the network and expect it to learn from that. We start by creating a dictionary composed of all the unique values in the data which we will turn into a vector where a 1 will appear depending on the current note. We can then vectorize the data based on the dictionary that we will build

from the midi file we have parsed it with the music21 library which gives us a ton of options to play around with later to modify this file and its contents however we want.

In our case, we need to transform the MIDI format from the files into something that will be much simpler to understand by a neural network. In other words, we need to make a big matrix and vectorize the notes.
We will also use one-hot-encoding to encode notes into a vector for ease of comprehension by the neural net

https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f


In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import tensorflow as tf

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network 
        creates a dictionary for the different notes so that these can be vectorized for an easy input into a NN
    """
    #batches of notes that we will feed into the NN
    sequence_length = 50

    # get all pitch names
    #sorts all of the pitches,note,chords etc that appear so we can make a dictionary for vectorizing notes
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers 
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # turns the integers corresponding to the notes in the dictionary to vectors of size = size(dictionary), (one hot encoding)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


### Initializing the LSTM network


Configuring Keras (and TF) so that the model can be run on a GPU available with google Colab

https://stackoverflow.com/questions/45662253/can-i-run-keras-model-on-gpu

In [0]:
from keras import backend

backend.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

    return model


In [0]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "trained_models/weights-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    history= model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)
    
    return history


### main loop

In [0]:
#notes = get_notes() should have been run above

n_vocab = len(set(notes))
print(n_vocab)

# Data formation
network_input, network_output = prepare_sequences(notes, n_vocab)
# Model architecture
model = create_network(network_input, n_vocab)
# Treaining
history = train(model, network_input, network_output)
# Ploting

plt.plot(history.history["acc"])
plt.show()

plt.plot(history.history["loss"])
plt.show()


208
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
24955/24955 [==============================] - 147s 6ms/step - loss: 4.2767 - acc: 0.0596
Epoch 2/100
24955/24955 [==============================] - 144s 6ms/step - loss: 4.2130 - acc: 0.0636
Epoch 3/100
24955/24955 [==============================] - 145s 6ms/step - loss: 4.2053 - acc: 0.0646
Epoch 4/100
24955/24955 [==============================] - 144s 6ms/step - loss: 4.1883 - acc: 0.0654
Epoch 5/100
24955/24955 [==============================] - 144s 6ms/step - loss: 4.2161 - acc: 0.0712
Epoch 6/100
24955/24955 [==============================] - 144s 6ms/step - loss: 4.0078 - acc: 0.0756
Epoch 7/100
24955/24955 [==============================] - 144s 6ms/step - loss: 3.9775 - acc: 0.0769
Epoch 8/100
24955/24955 [======================

In [0]:

plt.plot(history.history["loss"])
plt.show()

NameError: ignored

### Using the trained model to predict new notes

In [0]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

In [0]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 50
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [0]:
def load_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])

    # Load the weights to each node
    model.load_weights('trained_models/weights2.hdf5')

    return model


In [0]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    
    pattern = network_input[start]
    prediction_output = []

    # generate 1000 notes
    for note_index in range(750):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        
        
        index = numpy.argmax(prediction)
        
        
        
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [0]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        print(pattern)
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        
        #note is a rest
        
        elif pattern == "rest" :
            offset+= 0.5
        
        # check ifpattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        
        
        
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='generated_music/test_output10.mid')


In [0]:
with open('data/notes/note_tab', 'rb') as filepath:
    notes = pickle.load(filepath)

# Get all pitch names
pitchnames = sorted(set(item for item in notes))

print(pitchnames)

n_vocab = len(set(notes))
# Pre process
network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)

# Defing model architecture
model = load_network(normalized_input, n_vocab)

# Generate music
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)

# Create midi file
create_midi(prediction_output)

['0', '0.1.5', '0.2', '0.2.4.6', '0.2.4.6.9', '0.2.6', '0.2.6.7', '0.3', '0.3.5', '0.3.6', '0.3.6.8', '0.3.7', '0.4', '0.4.6', '0.4.7', '0.5', '0.6', '1.3.5.7', '1.4', '1.4.7', '1.4.7.9', '1.4.8', '1.5', '1.5.8', '1.6', '1.7', '10', '10.0', '10.0.3', '10.0.4', '10.1', '10.1.5', '10.11.0.1', '10.2', '10.2.4', '10.2.5', '10.3', '11', '11.0', '11.0.4', '11.1.4', '11.1.4.7', '11.2', '11.2.4', '11.2.4.7', '11.2.5', '11.2.5.7', '11.2.6', '11.3', '11.3.5', '11.3.6', '11.4', '2', '2.3', '2.3.7', '2.4', '2.4.7', '2.4.7.10', '2.4.7.8', '2.4.8', '2.5', '2.5.7', '2.5.8', '2.5.8.11', '2.5.9', '2.6', '2.6.9', '2.7', '2.7.8', '2.8', '3', '3.4', '3.4.6.9', '3.5', '3.5.10', '3.5.7', '3.5.7.9', '3.5.8.11', '3.6', '3.6.9', '3.7', '3.7.10', '3.7.11', '3.7.9', '3.9', '4', '4.10', '4.5', '4.5.9.0', '4.6.10', '4.6.9.0', '4.7', '4.7.10', '4.7.10.0', '4.7.11', '4.7.9', '4.8', '4.8.10', '4.8.11', '4.9', '5', '5.10', '5.11', '5.7', '5.7.11', '5.7.9.11', '5.8', '5.9', '5.9.0', '5.9.11', '6.10', '6.10.1', '6.11', 